# Data Wrangling

https://www.cbp.gov/newsroom/stats

https://www.ice.gov/detain/detention-management

https://www.uscis.gov/tools/reports-and-studies/immigration-and-citizenship-data

https://ohss.dhs.gov/topics/immigration#other-resources 


# Ideas
- Write about H1B, H2A, H2B, or other visas
- Write about detained individual counts at the border
- Write about detained individuals within the US
- Forecast immigration data
- Forecast impacts of policy changes

- Combine company visa information with their stock information
- Investigate if there are any correlations between stock information and visa information
- Can we make price movement predictions with visa information of publicly listed companies



In [49]:
# Imports
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
from yfinance import ticker
import json
from io import StringIO
import re
from rapidfuzz import process, fuzz
from joblib import Parallel, delayed

In [62]:
# Gets a list of all listed companies from SEC's EDGAR database
#"https://www.sec.gov/files/company_tickers.json"

with open('data/company_tickers.json', 'r') as f:
    data = json.load(f)
# companies = [entry["ticker"] for entry in response.values()]
# print(companies[:10])  # Print first 10 tickers

In [ ]:
### WORK IN PROGRESS ###

# def read_tsv(file_path):
#     """Reads a TSV file with UTF-8 encoding."""
#     with open(file_path, "r", encoding="utf-8") as f:
#         return pd.read_csv(StringIO(f.read()), delimiter="\t", low_memory=False)
    
# remove_terms = [
#     "INCORPORATED", "INC", "CORPORATION", "CORP", "NA", "N A", "ENTERTAINMENT",
#     "CONSULTING", "FINANCIAL", "FINANCIALS", "LOGISTICS", "INDUSTRIES", "MANAGEMENT",
#     "LTD", "LIMITED", "LLC", "LLP", "PLC", "GROUP", "GRP", "GR", "HOLDINGS", "COMPANY",
#     "CO", "LP", "LL", "PARTNERSHIP", "TECHNOLOGIES", "SYSTEMS", "GLOBAL", "ENTERPRISES",
#     "SERVICES", "COM", "COMMUNICATIONS", "SOLUTIONS", "INVESTMENT", "PHARMACEUTICALS"
# ]
# remove_terms_pattern = re.compile(r'\b(' + '|'.join(remove_terms) + r')\b')
    
# def remove_meta_characters(input_string):
#     input_string = str(input_string)
#     input_string = re.sub(r'\s+', ' ', input_string).strip() 
#     input_string = remove_terms_pattern.sub('', input_string)
#     input_string = re.sub(r'/[^/]+/', ' ', input_string)
#     input_string = input_string.replace('.', '').replace(',', '').replace('&', 'AND')
#     input_string = re.sub(r'\s+', ' ', input_string).strip() 
#     return input_string

# def find_best_match(employer, standard_titles, company_mapping, score_cutoff=75):
#     matches = process.extract(employer, standard_titles, scorer=fuzz.token_set_ratio)
#     valid_matches = [match for match in matches if match[1] >= score_cutoff]
#     if valid_matches:
#         best_match_name = max(valid_matches, key=lambda x: x[1])[0]
#         return best_match_name, company_mapping.get(best_match_name)
#     return None, None

# def parallel_find_best_match(employers, standard_titles, company_mapping, n_jobs=-1):
#     return Parallel(n_jobs=n_jobs, backend="threading")(
#         delayed(find_best_match)(emp, standard_titles, company_mapping) for emp in employers)

# titles = [company['title'].upper() for company in data.values()]
# standard_titles = [remove_meta_characters(title) for title in titles]
# standard_titles = list(set(standard_titles))

# company_mapping = {remove_meta_characters(company['title']): company['ticker'] for company in data.values()}

# visa_files = {"h1b": ['data/visas/h1b_2016_2009.csv', 'data/visas/h1b_2024_2017.csv']}

# df_visas = []
# for visa_type, files in visa_files.items():
#     df_list = [read_tsv(file) for file in files]
#     df = pd.concat(df_list, axis=0)
#     df["employer"] = df["Employer (Petitioner) Name"].str.upper().apply(remove_meta_characters)
#     df.loc[:, "type"] = visa_type
#     df_visas.append(df)
 
# df_visas_combined = pd.concat(df_visas)

# drop_cols = ['Line by line', 'Employer (Petitioner) Name', 'Initial Denial', 'Continuing Denial']
# df_visas_clean = df_visas_combined.drop(drop_cols, axis = 1)
# df_visas_clean.columns = df_visas_clean.columns.str.strip()

# int_columns = ["Fiscal Year", "Tax ID", "Petitioner Zip Code", "Initial Approval", "Continuing Approval"]

# df_visas_clean[int_columns] = df_visas_clean[int_columns].apply(pd.to_numeric, errors="coerce").astype("Int64")

# sum_data = df_visas_clean.groupby(["Fiscal Year", "employer"], as_index=False).agg({
#     "Tax ID": "first",  
#     "Industry (NAICS) Code": "first",  
#     "Petitioner City": "first",  
#     "Petitioner State": "first",  
#     "Petitioner Zip Code": "first",  
#     "Initial Approval": "sum",  
#     "Continuing Approval": "sum",  
#     "type": "first"  
# })

# matches = parallel_find_best_match(sum_data["employer"], standard_titles, company_mapping, n_jobs=10)

# matches_df = pd.DataFrame(matches, columns=["matched_company", "ticker"])
# sum_data_filter = pd.concat([sum_data, matches_df], axis=1)

# sum_data_filter_narm = sum_data_filter[sum_data_filter["matched_company"].notna()]

# final_sum = sum_data_filter_narm.groupby(["Fiscal Year", "ticker"], as_index=False).agg({
#     "matched_company": "first",
#     "Tax ID": "first",  
#     "Industry (NAICS) Code": "first",  
#     "Petitioner City": "first",  
#     "Petitioner State": "first",  
#     "Petitioner Zip Code": "first",  
#     "Initial Approval": "sum",  
#     "Continuing Approval": "sum",  
#     "type": "first"  
# })

# final_sum.to_csv("data/final_sum.csv", index=False)

# # This code chunk took about 75 minutes to run and complete. My laptop and I are not on speaking terms.

In [ ]:
h1b_df = pd.read_csv("data/final_sum.csv")
# Note: some large companies are omitted (AAPL, MRST) while others are most likely overestimated due to false positives (AAME, AABQ)
# I'll get around to fixing these issues after the midterm, though it should be sufficient for the proposal - Matt

# I no longer need the below chunks for debugging. Feel free to remove them if you feel the same. 

In [ ]:

h1b_2009_path = 'data/visas/h1b_2016_2009.csv'
h1b_2017_path = 'data/visas/h1b_2024_2017.csv'

h2a_2015_path = 'data/visas/h2a_2019_2015.csv'
h2a_2020_path = 'data/visas/h2a_2024_2020.csv'

h2b_2015_path = 'data/visas/h2b_2019_2015.csv'
h2b_2020_path = 'data/visas/h2b_2025_2020.csv'

#Had to convert all encoding to UTF8 via notepad++
def read_tsv(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        return pd.read_csv(StringIO(f.read()), delimiter="\t", low_memory=False)

# Read all files using the function
df_h1b_a = read_tsv(h1b_2009_path)
df_h1b_b = read_tsv(h1b_2017_path)

df_h2a_a = read_tsv(h2a_2015_path)
df_h2a_b = read_tsv(h2a_2020_path)

df_h2b_a = read_tsv(h2b_2015_path)
df_h2b_b = read_tsv(h2b_2020_path)

C:\Users\peter\AppData\Local\Temp\ipykernel_10120\3969071596.py:13: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(StringIO(f.read()), delimiter="\t")
C:\Users\peter\AppData\Local\Temp\ipykernel_10120\3969071596.py:13: DtypeWarning: Columns (9,11) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(StringIO(f.read()), delimiter="\t")
C:\Users\peter\AppData\Local\Temp\ipykernel_10120\3969071596.py:13: DtypeWarning: Columns (4,16,18) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(StringIO(f.read()), delimiter="\t")
C:\Users\peter\AppData\Local\Temp\ipykernel_10120\3969071596.py:13: DtypeWarning: Columns (16,20) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(StringIO(f.read()), delimiter="\t")


In [14]:
df_h1b = pd.concat([df_h1b_a, df_h1b_b], axis=0)

df_h2a = pd.concat([df_h2a_a, df_h2a_b], axis=0)

df_h2b = pd.concat([df_h2b_a, df_h2b_b], axis=0)


In [ ]:
print(df_h1b.columns)
print(df_h2a.columns)
print(df_h2b.columns)

In [ ]:
def remove_meta_characters(input_string):
    input_string = str(input_string)
    input_string = re.sub(r'\s+', ' ', input_string).strip() 
    input_string = re.sub(r'\b(INCORPORATED|INC|CORPORATION|CORP|NA|N A|LTD|LIMITED|LLC|LLP|PLC|GROUP|GRP|GR|HOLDINGS|COMPANY|CO|LP|PARTNERSHIP)\b',
                          '', str(input_string).upper())
    input_string = re.sub(r'/[^/]+/', ' ', input_string)
    input_string = input_string.replace('.', '').replace(',', '').replace('&', 'AND')
    input_string = re.sub(r'\s+', ' ', input_string).strip() 
    return input_string

In [ ]:
titles = [company['title'].upper() for company in data.values()]

standard_titles = [remove_meta_characters(title) for title in titles]

In [ ]:
df_h1b['employer'] = df_h1b['Employer (Petitioner) Name'].str.upper().apply(remove_meta_characters)
df_h2a['employer'] = df_h2a['Employer (Petitioner) Name'].str.upper().apply(remove_meta_characters)
df_h2b['employer'] = df_h2b['Employer (Petitioner) Name'].str.upper().apply(remove_meta_characters)

In [ ]:
print('H1B Crossover Count: ' + str(sum(df_h1b["employer"].isin(standard_titles))))
print('H2A Crossover Count: ' + str(sum(df_h2a["employer"].isin(standard_titles))))
print('H2B Crossover Count: ' + str(sum(df_h2b["employer"].isin(standard_titles))))


In [ ]:
df_h1b_listed = df_h1b[ df_h1b["employer"].isin(standard_titles) ].copy()
df_h1b_listed.loc[:, "type"] = 'h1b'

df_h2a_listed = df_h2a[ df_h2a["employer"].isin(standard_titles) ].copy()
df_h2a_listed.loc[:, "type"] = 'h2a'

df_h2b_listed = df_h2b[ df_h2b["employer"].isin(standard_titles) ].copy()
df_h2b_listed.loc[:, "type"] = 'h2b'



In [ ]:
print(df_h1b.columns)
print(df_h2a.columns)
print(df_h2b.columns)

In [ ]:
df_visas = pd.concat([df_h1b_listed, df_h2a_listed, df_h2b_listed])
df_visas.shape

h1b_drop_cols = ['Line by line', 'Employer (Petitioner) Name', 'Tax ID', 'Initial Denial', 'Continuing Denial', 'Initial Approval', 'Continuing Approval']
#Adding the columns does not work since they are currently strings, to_numeric not working due to commas
df_h1b_listed['tot'] = df_h1b_listed['Initial Approval'] + df_h1b_listed['Continuing Approval']
df_h1b_clean = df_h1b_listed.drop(h1b_drop_cols, axis = 1)
df_h1b_clean

In [ ]:
df_visas.columns


In [ ]:
drop_cols = ['Line by line', 'Employer (Petitioner) Name', 'Tax ID', 'Initial Denial', 'Continuing Denial', 'Index()', 'Consular_processed', 'New Employment Denial', 'Continuation Denial', 'Change with Same Employer Denial', 'New Concurrent Denial', 'Change of Employer Denial', 'Amended Denial', 'Consular_Processed', 'ETA Case Number']
df_visas_clean = df_visas.drop(drop_cols, axis = 1)

In [ ]:
df_visas_clean.head()

In [ ]:
import time
def fetch_stock_data(tickers):
    all_data = {}
    print('Downloading yfinance historical price data 2000/01/01 to 2025/03/16')
    num_downloaded = 0
    for ticker in tickers:
        try:
            data = yf.download(ticker, start="2000-01-01", end="2025-03-16", progress = False)
            all_data[ticker] = data
            num_downloaded +=1
            time.sleep(2.2)  # Sleep for 2 seconds between requests to avoid being blocked
        except Exception as e:
            print(f"Error downloading {ticker}: {e}")
            time.sleep(2.2)

    print('Successfully downloaded ' + str(num_downloaded/len(tickers)))
    #Save a csv for offline work
    sp500_data = pd.concat(all_data, axis=1)
    sp500_data.to_csv("sp500_data.csv")
    
    print("S&P 500 historical data saved to sp500_data.csv")
    return sp500_data

#Only run this once on your system
#Make sure NOT to track the sp500_data csv
#sp500csv made for offline work
#fetch_stock_data